In [1]:
import pandas as pd

In [5]:
building_data = pd.read_parquet("//Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/MA_baseline_metadata_and_annual_results.parquet")

# Heating

In [11]:
col_list = [
    "in.heating_setpoint",
    "in.heating_setpoint_has_offset",
    "in.heating_setpoint_offset_magnitude",
    "in.heating_setpoint_offset_period",
]

heating_data = building_data[col_list]

In [20]:
heating_data["in.heating_setpoint_offset_period"].value_counts()

in.heating_setpoint_offset_period
None                 6627
Night +5h             458
Night -5h             442
Night                 440
Night +2h             435
Night +1h             431
Night +4h             421
Night -4h             409
Night +3h             404
Night -1h             396
Night -3h             392
Night -2h             380
Day and Night +5h      39
Day and Night -2h      30
Day and Night -5h      30
Day and Night +4h      30
Day and Night +2h      29
Day and Night -3h      29
Day and Night -4h      28
Day and Night +1h      27
Day and Night -1h      26
Day and Night          26
Day and Night +3h      22
Day +4h                19
Day -2h                19
Day +3h                19
Day -1h                16
Day                    15
Day +2h                13
Day -4h                13
Day +1h                12
Day -3h                11
Day +5h                10
Day -5h                 9
Name: count, dtype: int64

In [ ]:
def generate_building_heating_timeseries(building_id: int, heating_data: pd.DataFrame) -> pd.DataFrame:
    """Generates a building's heating setpoint timeseries.

    Args:
        building_id (int): ID of the building
        heating_data (pd.DataFrame): Full building dataset

    Returns:
        pd.DataFrame: Building heating setpoint timeseries
    """
    building_data = heating_data[heating_data.index == building_id]

    # Generate empty timeseries for a full year (2018)
    timeseries = pd.date_range(start="2018-01-01", end="2018-12-31", freq="H")
    timeseries = pd.DataFrame(timeseries, columns=["timestamp"])

    timeseries["setpoint"] = int(building_data["in.heating_setpoint"][0:2])

    # Change values between 

    if building_data["in.heating_setpoint_has_offset"].values[0] == "Yes":
